In [1]:
import string
import numpy as np

In [2]:
vocab = string.digits + string.ascii_letters + ' ' + string.punctuation
vocab_size = len(vocab)

stoi = {c:i for c,i in zip(vocab, range(vocab_size))}
itos = {i:c for c,i in zip(vocab, range(vocab_size))}

def encode(text: str) -> list[int]:
    return [stoi[c] for c in text]

def decode(idxs: list[int]) -> str:
    return ''.join([itos[i] for i in idxs])

In [3]:
def onehot(n_classes: int, idx: int) -> np.ndarray:
    emb = np.zeros(n_classes)
    emb[idx] = 1.0
    return emb

def str2seq(text: str) -> np.ndarray:
    return np.array([onehot(vocab_size, i) for i in encode(text)])

def seq2str(seq: np.ndarray) -> str:
    return decode([int(np.argmax(v)) for v in seq])

In [4]:
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

In [5]:
def calculate_attention(states: np.ndarray, context: np.ndarray) -> np.ndarray:
    scores = softmax(states @ context)
    values = scores.reshape((scores.size, 1)) * states
    return np.sum(values, axis=0)

In [39]:
class EncoderRNN:
    def __init__(self, n_in: int, n_h: int) -> None:
        self.W_xh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.W_hh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.b_h: np.ndarray = np.zeros(n_h)
    
    def forward(self, sequence: np.ndarray) -> np.ndarray:
        states = np.zeros((len(sequence), self.b_h.size))
        context = np.zeros(len(self.W_hh))

        for i,x in enumerate(sequence):
            context = np.tanh(self.W_hh @ context + self.W_xh @ x + self.b_h)
            states[i] = context.copy()
        
        return (states, context)

In [41]:
class DecoderRNN:
    def __init__(self, n_in: int, n_h: int, n_out: int) -> None:
        self.W_oh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.W_hh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.b_h: np.ndarray = np.zeros(n_h)

        self.W_ho: np.ndarray = np.random.uniform(-1, 1, (n_out, 2*n_h))
        self.b_o: np.ndarray = np.zeros(n_out)
    
    def forward(self, states: np.ndarray, context: np.ndarray, t: int) -> np.ndarray:
        outputs = np.zeros((t, self.b_o.size))
        out = np.zeros(self.b_o.size)

        for i in range(t):
            context = np.tanh(self.W_hh @ context + self.W_oh @ out + self.b_h)
            attention = calculate_attention(states, context)
            ctx_att = np.concatenate((context, attention), axis=0)
            out = softmax(self.W_ho @ ctx_att + self.b_o)
            outputs[i] = out.copy()
            
        return outputs

In [30]:
class Seq2Seq:
    def __init__(self, n_in: int, n_h: int, n_out: int) -> None:
        self.encoder = EncoderRNN(n_in, n_h)
        self.decoder = DecoderRNN(n_in, n_h, n_out)

    def generate(self, sequence: np.ndarray, t: int) -> np.ndarray:
        states, context = self.encoder.forward(sequence)
        return self.decoder.forward(states, context, t)

In [49]:
model = Seq2Seq(vocab_size, 32, vocab_size)

in_text = 'the quick brown fox jumps over the lazy dog'
sequence = str2seq(in_text)

outputs = model.generate(sequence, 128)
out_text = seq2str(outputs)

print(out_text)

aFMyy{CF0t=et0\QQiy0\-GAyX`o4T|zO{'*YU3i{g0?gI?Q*#*hGaI-tA'CI:y=I?Q)'gi?h0Cet{CI0QC>04ybjt94TK?QQiyl\9cW4Ml{M4$9p03bAAXN&Mvy7C9}
